# Vision Transformer (An Image Is Worth $16\times 16$ Words: Transformers For Image Recognition An Scale)  
- 논문 리뷰

## Abstract  
- Transformer architecture는 NLP task에서 표준이 되었지만 Vision 분야에 대한 application은 제한적임  
- 이 논문에서는 이제 CNN에 대한 의존이 필요없으며 이미지 패치의 sequence를 바로 적용한 순수한 transformer가 이미지 분류 task 에서 가장 좋은 성능을 낸다는 것을 보일 것이다  
- ViT는 CNN 기반의 SOTA 모델과 비교해 우수한 성능을 내며 훈련할 때 CNN보다 훨씬 더 적은 resource를 필요로 함  

## Introduction  
- transformer에서 self attention 기반 architecture는 NLP에서 좋은 성능을 보이고 있음  
- transformer의 계산적인 효율성과 확장성 덕분에 100B 이상의 파라미터의 모델을 학습할 수 있게 됨  
- 엄청 빠른 훈련 속도 덕분에 무수히 많은 파라미터의 모델을 충분히 학습할 수 있다는 의미인 듯  
- 그러나 Vision 분야에서는 여전히 CNN이 우세하며 NLP에서 영감을 받아 CNN과 self attention을 결합하거나 CNN을 완전히 대체하는 연구들이 있었음  
- 그 후에 나온 모델들은 이론상으론 효과적이었으나 아직 하드웨어 가속기에서는 사용을 못함  
- 이미지 분류 task에서는 아직까지 ResNet 기반 모델이 SOTA  
- NLP의 transformer 성공에 영감을 받아 가능한 최소한의 수정으로 이미지를 standard transformer에 입력해 실험  
- 이를 위해 이미지를 패치로 분할하고 linear embedding sequence를 transformer에 입력  
- 즉, 패치를 NLP에서 token과 같이 다루는 것  
- 그리 크지 않은 사이즈의 ImageNet으로 훈련할 때 ResNet보다 조금 낮은 정확도를 출력  
- 이는 실망스러운 결과일 수 있으나 CNN에 내재하는 translation equivariance와 locality 같은 inductive bias가 transformer에는 존재하지 않는다는 점을 감안하면 충분히 좋은 성능임  
- 따라서 충분하지 않은 dataset으로 훈련하면 일반화가 잘 되지 않음   
- 그러나 이미지 14M~300M 장의 large scale dataset에서 학습한다면 말이 달라짐  
- large scale training이 inductive bias를 능가한다는 것을 찾았음  
- ViT는 충분한 scale로 사전 학습되고 적은 dataset으로 transfer learning을 할 때 좋은 결과를 얻음  
- ImageNet에서 88.55%, ImageNet-Real에서 90/72%, CIFAR-100에서 94.55%의 성능을 달성  

## Method  
- model design은 original Transformer와 비슷하게 사용  
- 이렇게 설정함으로써 NLP transformer architecture를 간단한 setup만으로 확장할 수 있음  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/ViT/figure/figure1.JPG?raw=true">

### Vision Transformer (ViT)  
- standard transformer는 token embedding의 $1D$ sequence를 input으로 받음  
- $2D$ image를 다루기 위해 $\textbf{x}\in\mathcal{R}^{H\times W\times C}$를 reshape시켜 $\textbf{x}\in\mathcal{R}^{N\times \left(P^2C\right)}$로 만듦  
- 이때 $\left(H, W\right)$는 이미지의 해상도, $C$는 이미지의 채널, $\left(P,P\right)$는 image patch의 해상도, $N=HW/P^2$이다  
- transformer는 모든 레이어들의 latent vector 사이즈가 $D$로 통일되어 있어 이러한 2차원 patch들을 다시 1차원으로 flatten하고 훈련 가능한 linear projection을 거쳐 $D$차원 벡터로 매핑시킴  
- 논문에서는 projection의 output을 patch embedding이라 함  
- BERT의 token과 유사하게 embedding된 patch의 sequence에 학습 가능한 embedding을 추가하며  
- Transformer encoder의 output은 식 4의 $\textbf{y}$로 표현할 수 있음 ($\textbf{y}$: image representation vector)  
- pre-training과 fine-tuning을 하면 classification head에 image representation vector가 부착됨  
- classification head에는 사전 훈련을 할 때는 하나의 hidden layer를 사용하며 fine-tuning을 할 때는 단일 linear layer로 구성된 MLP 구조를 따름  
- 그리고 위치 정보를 유지하기 위해 position embedding을 추가함  
- 논문에서 advanced한 $2D$-aware position embedding이 유의미한 성능을 얻지 못해 $1D$ position embedding을 사용함  
- 이렇게 구성된 최종 embedding 벡터의 sequence를 encoder로 입력함  

$$\textbf{z}=\left[\textbf{x};\textbf{x}_p^1E;\textbf{x}_p^2E;\cdots;\textbf{x}_p^NE\right]+E_{pos},\qquad E\in\mathcal{R}^{\left(P^2C\right)\times D},\, E_{pos}\in\mathcal{R}^{\left(N+1\right)\times D}\quad (1)$$  
$$\textbf{z}_l'=MSA\left(LN\left(\textbf{z}_{l-1}\right)\right)+\textbf{z}_{l-1},\qquad l=1,\cdots,L\quad (2)$$  
$$\textbf{z}_l=MLP\left(LN\left(\textbf{z}_l'\right)\right)+\textbf{z}_l',\qquad l=1,\cdots,L \quad (3)$$  
$$\textbf{y}=LN\left(\textbf{z}_L^0\right) \quad (4)$$  

- Transformer encoder는 MSA layer와 MLP block으로 구성됨  
- LN(Layer Norm)은 모든 block 전에 적용되며 block 이후에는 residual connection이 적용됨  
- 이때 MLP는 GELU 활성화 함수를 사용  

#### Inductive Bias  
- Vision Transformer는 CNN보다 inductive bias가 훨씬 더 적음  
- CNN에서는 conv layer가 위치 정보를 잘 잡아낸다는 특징이 있음  
- ViT에서는 MLP layer에서만 이러한 특징이 있으며 self attention layer에는 global한 특징이 있음  
- 즉, 이미지 전체에 대한 정보를 보며 patch 사이의 spatial relation에 대한 학습이 필요함  

#### Hybrid Architecture  
- raw image patch를 대신해 input sequence는 CNN의 feature map으로 만들 수 있음  
- 이러한 Hybrid model에서 patch embedding projection E는 CNN의 feature map에서 추출된 patch들에 적용  

### Fine-Tuning and Higher Resolution  
- 일반적으로 ViT는 large dataset에서 사전 훈련을 시킨 후 fine-tuning을 통해 transfer learning을 진행  
- 이를 위해선 pre-trained된 prediction head를 제거하고 $D\times K$ feed forward layer를 붙임  
- 이때 $K$는 클래스 갯수  
- 몇몇 연구에서는 사전 훈련된 dataset보다 더 높은 해상도의 dataset으로 fine-tuning을 하는 것이 더 beneficial하단 사실을 밝혀냄  
- 더 높은 해상도의 이미지를 처리할 때 patch size는 유지하며 sequence vector의 길이만 늘리는 방식으로 진행  
- ViT는 임의의 sequence vector 길이를 다룰 수 있지만 pre-trained position embedding은 의미를 잃게 됨  
- 따라서 original image의 위치 정보에 따라 pre-trained position embedding의 $2D$ interpolation을 수행  
- 이러한 해상도 조정과 patch 추출이 이미지의 2차원 구조에 대한 inductive bias가 수동으로 주입됨  

## Experiments  
- 논문에서는 ResNet, ViT 그리고 하이브리드 모델의 성능을 평가  
- 각 모델의 data 필요조건을 이해하기 위해 다양한 사이즈의 dataset으로 사전 훈련을 했으며 다양한 benchmark task에 대해 평가를 진행  
- 모델의 사전 훈련 계산 비용을 고려하면 ViT가 가장 좋은 성능을 보임  
- 또한 더 적은 사전 훈련 비용으로 대부분의 benchmask에서 SOTA를 달성  

### Setup  
#### Datasets  
- 논문에서는 모델의 확장성을 연구하기 위해 ImageNet, ImageNet-21K, JFT 등 large scale의 다양한 dataset으로 pre-trained한 후 benchmask dataset에 대해 transfer learning을 시켜 성능을 측정함  
- benchmark dataset으로는 ImageNet-Real, CIFAR-10/100, Oxford-IIIT Pets, Oxford Flowers-102 등을 이용  

#### Model Variants  
- BERT를 기반으로 ViT 모델을 구축  
- Base 모델, Large 모델은 BERT와 거의 동일  
- Huge 모델은 더 큰 규모의 모델  
- 이때 ViT-L/16이 의미하는 것은 ViT Large model, $16\times 16$ patch size  
- transformer의 sequence 길이는 patch size와 반비례하므로 patch size가 작을수록 계산 비용 증가  
- CNN 기반 모델은 ResNet을 사용했으며 BN layer 대신 Group Normalization layer과 standardized conv을 사용  
- 이렇게 수정함으로써 transfer learning의 성능을 더 향상시킴  
- 이 모델을 ResNet(BiT)로 표기  
- 하이브리드 모델에 대해서는 CNN의 중간 feature map들을 ViT에 $1\times 1$ patch로 쪼개서 넣음  

#### Training & Fine-tuning  
- 사전 훈련 시에는 모든 모델을 Adam으로 훈련시켰으며 $\beta_1=0.9$, $\beta_2=0.999$, batch size는 4096, weight decay는 0.1, scheduler로는 linear lr warm up을 사용  
- fine-tuning 시에는 SGD momentum, batch size는 512를 사용  

#### Metrics  
- 논문에서 downstream dataset에 대한 결과를 few-shot 혹은 fine-tuning accuracy로 평가  
- fine-tuning accuracy는 각 모델의 성능을 평가  
- 일반적인 accuracy를 측정하는 방식과 같음  
- few-shot accuracy는 fine-tuning이 너무 고비용인 경우에 사용  

### Comparison to State Of The Art  
- 우리의 ViT-H/14와 ViT-L/16을 가지고 CNN 기반 SOTA 모델인 ResNet의 BiT와 EfficientNet 기반 Noisy Student 모델과 성능 비교   
- TPUv3-core-days는 사전 훈련하는데 소요된 일 수와 TPU 코어 수를 곱한 수치를 의미  

<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/ViT/figure/table2.JPG?raw=true">  

- JFT-300M에서 사전 훈련된 ViT-L/16 model은 모든 task에서 BiT-L model을 능가함  
- 또한 훈련하는데 훨씬 더 적은 computation resource를 필요로 함  
- 더 큰 모델인 ViT-H/14 model도 대부분의 dataset에서 가장 좋은 성능을 보임  
- 흥미롭게도 ViT는 현재 SOTA model보다 계산량이 훨씬 더 적음 

### Pre-Training Data Requirements  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/ViT/figure/figure3.JPG?raw=true" width=500>  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/ViT/figure/figure4.JPG?raw=true" width=500>  

- ViT는 inductive bias가 ResNet보다 훨씬 더 적음에도 불구하고 훨씬 뛰어난 성능을 보임  
- dataset의 사이즈가 얼마나 중요한지 논문에서 다음 2가지의 실험을 진행함  
    - 1. dataset의 사이즈를 늘리면서 사전 훈련  
        - 적은 dataset에도 좋은 성능을 얻기 위해 weight decay, dropout, label smoothing을 진행  
        - ImageNet에서 사전 훈련을 했을 때, ViT-Large model이 ViT-base model보다 좋지 않은 성능을 보임  
        - ImageNet-21K일 때도 마찬가지  
        - 오직 JFT-300M dataset으로 사전 훈련을 했을 때만 ViT의 benefit을 확인할 수 있음  
        
    - 2. JFT-300M의 랜덤 subset들에 대해서도 사전 훈련  
        - 계산 비용을 절약하기 위해 metric으로 few-shot accuracy를 사용  
        - 적은 dataset에 대해서든 사전 훈련 시 BiT가 ViT보다 성능이 좋았음  
        - 또한 ViT가 over-fitting되는 경향이 있음  
        - 이 결과로 적은 dataset에 대해서는 convolutional inductive bias가 꽤 유용하지만 큰 dataset에 대해서는 의미있는 패턴을 학습하는 것만으로도 충분하거나 더 beneficial하다는 직관을 얻었음

### Scaling Study  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/ViT/figure/figure5.JPG?raw=true">  

- ViT는 성능 vs 계산 비용의 trade-off에서 ResNet을 압도함  
- 동일한 성능을 달성하는데 드는 계산 비용이 2~4배 더 적음  
- 두 번째로 적은 계산 비용 측면에서는 ViT보다 하이브리드 모델이 더 성능이 좋음  
- 이 차이는 모델이 커짐에 따라 사라지긴 함  
- 이 결과의 놀라운 점은 임의의 사이즈의 ViT가 convolutional local feature processing이 도움을 줄 수 있다는 것을 기대할 수 있기 때문

### Inspecting Vision Transformer  
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/ViT/figure/figure7.JPG?raw=true">
<img src = "https://github.com/Sangh0/Vision-Transformer/blob/main/ViT/figure/figure6.JPG?raw=true">

- ViT가 이미지 데이터를 어떻게 처리하는지 이해하기 위해 내부의 representation을 분석  
- ViT의 첫 번째 레이어는 flatten된 패치들을 저차원의 공간으로 linearly projection을 진행  
- 위 그림 왼쪽을 보면 학습된 embedding filter의 주성분들을 보여주고 있음  
- 이 성분들은 각 패치의 자세한 구조의 저차원의 representation을 위한 basis function과 비슷하다고 볼 수 있음  
- projection 이후에 학습된 position embedding은 patch representation에 추가됨  
- 위 그림 가운데에서 모델이 position embedding의 유사도 내에서 이미지 내부의 거리 개념을 인코딩하는 방법을 배운다는 것을 보여줌  
- 즉, 가까운 패치들은 유사한 position embedding을 가짐
- self-attention은 ViT가 이미지 전체의 정보를 통합해 사용할 수 있게 함  
- 논문에서는 네트워크가 이 수용력을 얼마나 이용하는지 연구함  
- attention distance는 CNN의 receptive field와 비슷한 개념  
- 논문에서는 몇몇의 attention head가 가장 낮은 레이어의 이미지 대부분을 attend함을 찾아냄  
- 이는 정보를 global하게 통합하는 기능이 있다는 것을 의미

# Conclusion  
- Computer Vision에서 self attention을 사용한 이전 연구들과 달리 inductive bias를 강제로 추가하지 않음  
- 대신, 이미지를 NLP의 transformer처럼 각 패치를 하나의 sequence처럼 이해하고 다뤘음  
- 이 간단하고 확장 가능한 전략은 large dataset에 대해 사전 훈련을 할 때 굉장히 성능이 좋음  
- 따라서 ViT는 SOTA를 달성하며 사전 훈련 비용이 상대적으로 저렴  
- 그러나 아직 개선해야 할 것들이 남음  
    - 1. ViT를 detection or segmentation task에 적용  
    - 2. self-supervised pre-training method 향상  
    - 3. 성능 향상을 위한 ViT의 추가적인 확장